In [16]:
import numpy as np
import pandas as pd

data = pd.read_csv("./DATA/training.csv", encoding='ISO8859-2')
data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,\nMore\nI can't make any real suggestions on i...,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
127652,ffe8b9316245be30,The numbers in parentheses are the additional ...,0,0,0,0,0,0
127653,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
127654,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
127655,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [18]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=0.1,
                        max_features=5000)

X = count.fit_transform(data['comment_text'].values)
X

<127657x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 2438814 stored elements in Compressed Sparse Row format>

In [19]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=6, learning_method='batch')
X_topics = lda.fit_transform(X)
X_topics

array([[0.00500073, 0.21293181, 0.00500158, ..., 0.41530841, 0.12798724,
        0.00500078],
       [0.00833741, 0.21057997, 0.00833359, ..., 0.00833425, 0.28821669,
        0.00833456],
       [0.50640188, 0.25950067, 0.00526411, ..., 0.0052649 , 0.00526428,
        0.00526417],
       ...,
       [0.02000443, 0.02001239, 0.02000259, ..., 0.81995693, 0.02000023,
        0.02000996],
       [0.01428784, 0.58317635, 0.01428603, ..., 0.01428689, 0.01428739,
        0.01428687],
       [0.80702771, 0.00555693, 0.00555584, ..., 0.00555608, 0.00555647,
        0.0055561 ]])

In [26]:
n_top_words = 10
feature_names = count.get_feature_names()
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1: -1]]))

Topic 1:
people know time did think right good want say make
Topic 2:
think know ve thanks list use link work good articles
Topic 3:
lol fat school redirect city jew war country states state
Topic 4:
fuck utc hate nigger gay 2005 ass 2006 block 2007
Topic 5:
deletion deleted speedy tag notable subject articles criteria guidelines note
Topic 6:
think section wp time point does make way good discussion
Topic 7:
image edit user use blocked editing copyright edits fair pages
Topic 8:
hi shit fucking thanks sex know dont moron game comment
Topic 9:
help thank welcome pages edit articles style ask thanks hope
Topic 10:
sources source http com www english reliable does language book


D:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [41]:
y_columns = data.columns[2:]
models = []

In [42]:
from sklearn.model_selection import train_test_split
for y_column in y_columns:
    y = data[y_column]
    X_train, X_valid, y_train, y_valid = train_test_split(X_topics, y, test_size=0.25, stratify=y)
    from lightgbm.sklearn import LGBMClassifier
    from sklearn.metrics import f1_score
    lgb =LGBMClassifier(max_depth=10, num_leaves=500,boosting_type='dart')
    lgb.fit(X_train, y_train)
    y_valid_pred = lgb.predict(X_valid)
    models.append(lgb)
    print("%s f1_score: %.2f" % (y_column, f1_score(y_valid, y_valid_pred) * 100))

toxic f1_score: 34.83
severe_toxic f1_score: 10.50
obscene f1_score: 29.79
threat f1_score: 0.00
insult f1_score: 22.11
identity_hate f1_score: 3.30


In [39]:
test = pd.read_csv("./DATA/test.csv", encoding='ISO8859-2')
test_x = count.transform(test['comment_text'].values)

In [66]:
test_topics = lda.transform(test_x)
result = test['id']
for (idx, column) in enumerate(y_columns):
    y_test = models[idx].predict(test_topics)
    result = pd.concat([result, pd.DataFrame(y_test, columns=[column], index=test.index)], axis=1)

In [68]:
result.to_csv("_results.csv", index=None)